In [1]:
!pip install ortools

You should consider upgrading via the 'C:\Users\jose.velasco\Anaconda3\envs\ES_GEA\python.exe -m pip install --upgrade pip' command.


In [12]:
# [START import]
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
import pandas as pd
import numpy as np
import json

In [3]:
# Import Distance Matrix 
df_distance = pd.read_excel('df_distance_matrix.xlsx', index_col=0)
df_distance

,col0,col1,col2,col3,col4,col5,col6,col7,col8,col9,col10,col11,col12,col13,col14,col15,col16
row0,0,548,776,696,582,274,502,194,308,194,536,502,388,354,468,776,662
row1,548,0,684,308,194,502,730,354,696,742,1084,594,480,674,1016,868,1210
row2,776,684,0,992,878,502,274,810,468,742,400,1278,1164,1130,788,1552,754
row3,696,308,992,0,114,650,878,502,844,890,1232,514,628,822,1164,560,1358
row4,582,194,878,114,0,536,764,388,730,776,1118,400,514,708,1050,674,1244
row5,274,502,502,650,536,0,228,308,194,240,582,776,662,628,514,1050,708
row6,502,730,274,878,764,228,0,536,194,468,354,1004,890,856,514,1278,480
row7,194,354,810,502,388,308,536,0,342,388,730,468,354,320,662,742,856
row8,308,696,468,844,730,194,194,342,0,274,388,810,696,662,320,1084,514
row9,194,742,742,890,776,240,468,388,274,0,342,536,422,388,274,810,468


In [4]:
# Transform to Numpy Array
distance_matrix = df_distance.to_numpy()
distance_matrix

array([[   0,  548,  776,  696,  582,  274,  502,  194,  308,  194,  536,
         502,  388,  354,  468,  776,  662],
       [ 548,    0,  684,  308,  194,  502,  730,  354,  696,  742, 1084,
         594,  480,  674, 1016,  868, 1210],
       [ 776,  684,    0,  992,  878,  502,  274,  810,  468,  742,  400,
        1278, 1164, 1130,  788, 1552,  754],
       [ 696,  308,  992,    0,  114,  650,  878,  502,  844,  890, 1232,
         514,  628,  822, 1164,  560, 1358],
       [ 582,  194,  878,  114,    0,  536,  764,  388,  730,  776, 1118,
         400,  514,  708, 1050,  674, 1244],
       [ 274,  502,  502,  650,  536,    0,  228,  308,  194,  240,  582,
         776,  662,  628,  514, 1050,  708],
       [ 502,  730,  274,  878,  764,  228,    0,  536,  194,  468,  354,
        1004,  890,  856,  514, 1278,  480],
       [ 194,  354,  810,  502,  388,  308,  536,    0,  342,  388,  730,
         468,  354,  320,  662,  742,  856],
       [ 308,  696,  468,  844,  730,  194,  194

In [40]:
# Create dictionnary with data
data = {}
data['distance_matrix'] = distance_matrix
print("{:,} destinations".format(len(data['distance_matrix'][0]) - 1))



16 destinations


In [7]:
# Create Dataframe of Matrix Distance
def create_excel(data):
    n_col = len(data['distance_matrix'][0])
    n_row = len(data['distance_matrix'])
    list_row = ['row' + str(i) for i in range(n_row)]
    list_col = ['col' + str(i) for i in range(n_row)]

    matrix = np.array(data['distance_matrix'])
    df = pd.DataFrame(data=matrix, index=list_row, columns=list_col)
    df.to_excel('df_distance_matrix.xlsx')

In [41]:
# Orders quantity (Boxes)
data['demands'] = [0, 1, 1, 2, 4, 2, 4, 8, 8, 1, 2, 1, 2, 4, 4, 8, 8]
# Vehicles Capacities (Boxes)
data['vehicle_capacities'] = [15, 15, 15, 15]
# Fleet informations
# Number of vehicles
data['num_vehicles'] = 4
# Location of the depot
data['depot'] = 0

In [60]:
list_distance = []

for i in range(0, len(data['distance_matrix'])):
    for j in range(0, len(data['distance_matrix'])): 
        list_distance.append({'NodeI':i, 'NodeJ':j, 'Distance':int(data['distance_matrix'][i][j])})

list_demands = []
for i in range(0, len(data['demands'])):
    list_demands.append({'Node':int(i), 'Demand':int(data['demands'][i]) })

list_vehicles = []

for i in range(0, len(data['vehicle_capacities'])):
    list_vehicles.append({'Vehicle':int(i), 'Capacity':int(data['vehicle_capacities'][i]) })



[{'Vehicle': 0, 'Capacity': 15},
 {'Vehicle': 1, 'Capacity': 15},
 {'Vehicle': 2, 'Capacity': 15},
 {'Vehicle': 3, 'Capacity': 15}]

In [61]:
data_json = data.copy()
data_json['distance_matrix'] = list_distance
data_json['DistanceMatrix'] = data_json.pop('distance_matrix')
data_json['Demands'] = list_demands
data_json.pop('demands')
data_json['VehicleCapacities'] = list_vehicles
data_json.pop('vehicle_capacities')
data_json['NumVehicles'] = data_json.pop('num_vehicles')
data_json['Depot'] = data_json.pop('depot')

{'DistanceMatrix': [{'NodeI': 0, 'NodeJ': 0, 'Distance': 0},
  {'NodeI': 0, 'NodeJ': 1, 'Distance': 548},
  {'NodeI': 0, 'NodeJ': 2, 'Distance': 776},
  {'NodeI': 0, 'NodeJ': 3, 'Distance': 696},
  {'NodeI': 0, 'NodeJ': 4, 'Distance': 582},
  {'NodeI': 0, 'NodeJ': 5, 'Distance': 274},
  {'NodeI': 0, 'NodeJ': 6, 'Distance': 502},
  {'NodeI': 0, 'NodeJ': 7, 'Distance': 194},
  {'NodeI': 0, 'NodeJ': 8, 'Distance': 308},
  {'NodeI': 0, 'NodeJ': 9, 'Distance': 194},
  {'NodeI': 0, 'NodeJ': 10, 'Distance': 536},
  {'NodeI': 0, 'NodeJ': 11, 'Distance': 502},
  {'NodeI': 0, 'NodeJ': 12, 'Distance': 388},
  {'NodeI': 0, 'NodeJ': 13, 'Distance': 354},
  {'NodeI': 0, 'NodeJ': 14, 'Distance': 468},
  {'NodeI': 0, 'NodeJ': 15, 'Distance': 776},
  {'NodeI': 0, 'NodeJ': 16, 'Distance': 662},
  {'NodeI': 1, 'NodeJ': 0, 'Distance': 548},
  {'NodeI': 1, 'NodeJ': 1, 'Distance': 0},
  {'NodeI': 1, 'NodeJ': 2, 'Distance': 684},
  {'NodeI': 1, 'NodeJ': 3, 'Distance': 308},
  {'NodeI': 1, 'NodeJ': 4, 'Distan

In [65]:
with open('data.json', 'w') as fp:
    json.dump(data_json, fp, indent=4)

#json.dumps(data_json, sort_keys=True, indent=4)

In [7]:
def distance_callback(from_index, to_index):
    """Returns the distance between the two nodes."""
    # Convert from routing variable Index to distance matrix NodeIndex.
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return data['distance_matrix'][from_node][to_node]

In [10]:
def demand_callback(from_index):
    """Returns the demand of the node."""
    # Convert from routing variable Index to demands NodeIndex.
    from_node = manager.IndexToNode(from_index)
    return data['demands'][from_node]

In [16]:
# Create the routing index manager.
manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                       data['num_vehicles'],
                                       data['depot'])

# Create Routing Model
routing = pywrapcp.RoutingModel(manager)

# Create and register a transit callback.
transit_callback_index = routing.RegisterTransitCallback(distance_callback)

# Define cost of each arc.
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

# Add Capacity constraint.
demand_callback_index = routing.RegisterUnaryTransitCallback(
    demand_callback)
routing.AddDimensionWithVehicleCapacity(demand_callback_index,
    0,  # null capacity slack
    data['vehicle_capacities'],  # vehicle maximum capacities
    True,  # start cumul to zero
    'Capacity')

# Setting first solution heuristic.
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
search_parameters.local_search_metaheuristic = (routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
search_parameters.time_limit.FromSeconds(1)

# Solve the problem.
solution = routing.SolveWithParameters(search_parameters)

In [18]:
solution.

<ortools.constraint_solver.pywrapcp.Assignment; proxy of <Swig Object of type 'operations_research::Assignment *' at 0x00000258676AF900> >

In [13]:
if solution:
    total_distance = 0
    total_load = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for driver {}:\n'.format(vehicle_id)
        route_distance = 0
        route_load = 0
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_load += data['demands'][node_index]
            plan_output += ' {0} Parcels({1}) -> '.format(node_index, route_load)
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
        plan_output += ' {0} Parcels({1})\n'.format(manager.IndexToNode(index),
                                                 route_load)
        plan_output += 'Distance of the route: {} (m)\n'.format(route_distance)
        plan_output += 'Parcels Delivered: {} (parcels)\n'.format(route_load)
        print(plan_output)
        total_distance += route_distance
        total_load += route_load
    print('Total distance of all routes: {:,} (m)'.format(total_distance))
    print('Parcels Delivered: {:,}/{:,}'.format(total_load, sum(data['demands'])))
else:
    print('No Solution')

Route for driver 0:
 0 Parcels(0) ->  4 Parcels(4) ->  3 Parcels(6) ->  1 Parcels(7) ->  7 Parcels(15) ->  0 Parcels(15)
Distance of the route: 1552 (m)
Parcels Delivered: 15 (parcels)

Route for driver 1:
 0 Parcels(0) ->  14 Parcels(4) ->  16 Parcels(12) ->  10 Parcels(14) ->  9 Parcels(15) ->  0 Parcels(15)
Distance of the route: 1552 (m)
Parcels Delivered: 15 (parcels)

Route for driver 2:
 0 Parcels(0) ->  12 Parcels(2) ->  11 Parcels(3) ->  15 Parcels(11) ->  13 Parcels(15) ->  0 Parcels(15)
Distance of the route: 1552 (m)
Parcels Delivered: 15 (parcels)

Route for driver 3:
 0 Parcels(0) ->  8 Parcels(8) ->  2 Parcels(9) ->  6 Parcels(13) ->  5 Parcels(15) ->  0 Parcels(15)
Distance of the route: 1552 (m)
Parcels Delivered: 15 (parcels)

Total distance of all routes: 6,208 (m)
Parcels Delivered: 60/60
